# 1. **Installing**


In [81]:
%pip install torch torchvision matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 580.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.3 MB/s eta 0:00:00
Reading package lis

# 2. **Neural Networks**

Neural networks comprise of layers/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In [88]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        # Convert each 28x28 image into a contiguous array
        # of 1x784 pixel values
        self.flatten = nn.Flatten()

        self.linear_relu_stack = nn.Sequential(
            # Applies a linear transformation on the input
            # using its stored weights and biases
            nn.Linear(28*28, 512),

            # Non-linear activations are what create the complex
            # mappings between the model’s inputs and outputs.
            # They are applied after linear transformations to
            # introduce nonlinearity, helping neural networks learn
            # a wide variety of phenomena.
            nn.ReLU(),

            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# We want to be able to train our model on a hardware
# accelerator like the GPU or MPS, if available.
device = "cpu"
if torch.cuda.is_available():
  device = "cuda"
elif torch.backends.mps.is_available():
  device = "mps"

# Create an instance of NeuralNetwork, and move it to the device
model = NeuralNetwork().to(device)

X = torch.rand(1, 28, 28, device=device)
logits = model(X)

# The last linear layer of the neural network returns logits -
# raw values in [-infty, infty] - which are passed to the
# nn.Softmax module. The logits are scaled to values [0, 1]
# representing the model’s predicted probabilities for each class.
pred_probab = nn.Softmax(dim=1)(logits)

y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

# Many layers inside a neural network are parameterized,
# i.e. have associated weights and biases that are optimized
# during training. Subclassing nn.Module automatically tracks
# all fields defined inside your model object, and makes all
# parameters accessible using your model’s parameters() or
# named_parameters() methods.
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Predicted class: tensor([8])
Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-2.9635e-02,  1.1998e-02, -3.5152e-02,  ...,  5.0900e-03,
          2.0663e-02, -6.7196e-03],
        [ 9.0518e-05,  3.4993e-03, -6.4485e-03,  ..., -1.4220e-02,
          2.3647e-02,  2.1601e-03]], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0079, -0.0309], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0137, -0.0362, -0.0189,  ...,  0.0031, -0.0341, -0.0125],
        [ 0.0026,  0.0343, -0.0270,  ..., -0.0001,  